In [ ]:
import pandas as pd
import numpy as np
from dashboard.src.common.automatic_cost_calc import cost_calculation_from_energysystem
import traceback
from oemof import solph
import plotly.graph_objects as go
import os

try:
    wpath = os.path.join(os.getcwd(), "data", "simulations", "717afad441e440b482b0a79b7fa0de03", "dumps")
    print(wpath)

    es = solph.EnergySystem()
    es.restore(dpath=wpath, filename="config.dump")
except FileNotFoundError as e:
    traceback.print_exc()
    es = None
except Exception as e:
    traceback.print_exc()
    es = None

if es is not None:
    investment_data = cost_calculation_from_energysystem(es)

    busses = []
    bus_figures = []

    investment_elements = []
    cost_elements = []
    cost_elements_2 = []
    cost_elements_3 = []
    emissions_elements = []

    results = es.results["main"]

    flows = [x for x in results.keys() if x[1] is not None]
    nodes = [x for x in results.keys() if x[1] is None]

    for flow in [x for x in flows if hasattr(results[x]["scalars"], "invest") and isinstance(x[1], solph.Bus)]:
        if type(flow[0]) is not solph.components.GenericStorage:
            UNIT = ("MW", "kW")
        else:
            UNIT = ("MWh", "kWh")

    for node in es.nodes:
        if isinstance(node, solph.Bus):
            busses.append(node)

    bus_graph = []
    for bus in busses:
        graph_data = {}

        fig = go.Figure()
        for t, g in solph.views.node(es.results["main"], node=bus)["sequences"].items():
            idx_asset = abs(t[0].index(bus) - 1)
            time_series = np.nan_to_num(g.values[:8760]) * pow(-1, idx_asset)

            if idx_asset > 0:
                graph_data[str(t[0][1])] = time_series
            else:
                graph_data[str(t[0][0])] = time_series

        dataframe = pd.DataFrame(graph_data, index=es.timeindex[:8760])
        for column in dataframe.columns:
            fig.add_trace(
                go.Scatter(
                    x=dataframe.index,
                    y=dataframe[column],
                    mode='lines',  # 'lines' or 'markers'
                    name=column
                )
            )

        fig.update_layout(
            title=f"{bus.label}",
            xaxis_title="Time",
            yaxis_title="MWh",
            legend_title="Component"
        )

        fig.show()
